In [2]:
import numpy as np
import anndata as ad
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import anndata
import seaborn as sns
import scvelo as scv 
from scipy.stats import zscore
from dtne import *

In [169]:
adata = filtered_l6ct_PC1_shifted

# 设置 slice 分类
slice_categories = ['NR1_2', 'NR2_2', 'DR1_2', 'DR2_2']

# Step 1: 分别展示每个 slice 的散点图，并设置对应 colorbar 的最大值和最小值
fig, axes = plt.subplots(2, 2, figsize=(4, 2))

for i, slice_category in enumerate(slice_categories):
    ax = axes[i//2, i%2]  # 获取当前子图的位置
    # 筛选当前 slice 的数据
    subset = adata[adata.obs['slice'] == slice_category, :]
    
    # 计算当前 slice 的 PC1 的最小值和最大值
    pc1_slice_min = subset.obs['PC1'].min()
    pc1_slice_max = subset.obs['PC1'].max()
    
    # 绘制当前 slice 的散点图，colorbar 范围为该 slice 的 PC1 最小值和最大值
    scv.pl.scatter(subset, basis='stereo_rotated', color='PC1', cmap='viridis', 
                   vmin=pc1_slice_min, vmax=pc1_slice_max,  # 设置当前 slice 的 colorbar 范围
                    ax=ax, show=False, 
                   size=40, title=slice_category)
    
    # 获取 colorbar 对象
    cbar = ax.collections[0].colorbar
    # 设置 colorbar 的 ticks 为当前 slice 的最小值和最大值
    cbar.set_ticks([pc1_slice_min, pc1_slice_max])
    
    # 设置 colorbar 的标签
    cbar.set_label('PC1', fontsize=8)  # 设置 colorbar 标签字体大小

    # 调整刻度标签的字体大小
    cbar.ax.tick_params(labelsize=8)  # 设置刻度标签的字体大小
    # 设置 title 的字体大小
    ax.set_title(slice_category, fontsize=10)  # 设置 title 字体大小

# 调整子图布局
plt.subplots_adjust(hspace=0.3, wspace=0.3, bottom=0.2)
# 显示图形
#plt.show()
# 保存为 PDF
output_path = "F:/spatial/V2L/Figure/V2L_l6ct_PC1.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")






Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
PDF saved as F:/spatial/V2L/Figure/V2L_l6ct_PC1.pdf


In [66]:
sns.kdeplot(
    data=filtered_NR_l6ct_PC1.obs, x="digital_layer", y="PC1",
    fill=True, thresh=0, levels=10, cmap="mako",clip=((7, 11), (-4, 4)))
plt.text(0.95, 0.95, 'NR', ha='center', va='center', transform=plt.gca().transAxes, color='white', fontsize=12)
#plt.show()
output_path = "F:/spatial/V2L/Figure/V2L_l6ct_NR_digital_layer&PC1.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/V2L/Figure/V2L_l6ct_NR_digital_layer&PC1.pdf


In [67]:
sns.kdeplot(
    data=filtered_DR_l6ct_PC1.obs, x="digital_layer", y="PC1",
    fill=True, thresh=0, levels=10, cmap="mako",clip=((7, 11), (-4, 4)))
plt.text(0.95, 0.95, 'DR', ha='center', va='center', transform=plt.gca().transAxes, color='white', fontsize=12)
#plt.show()
output_path = "F:/spatial/V2L/Figure/V2L_DR_l6ct_digital_layer&PC1.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/V2L/Figure/V2L_DR_l6ct_digital_layer&PC1.pdf


In [49]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# 假设V2L_l6ct.obs中有 "PC1" 和 "condition" 列
data = V2L_l6ct.obs['PC1']
condition_list = V2L_l6ct.obs['condition'].unique()

# 创建一个空字典来保存每个condition的FWHM
fwhm_dict = {}

# 使用sns.displot来绘制KDE图，返回一个FacetGrid对象
g = sns.displot(V2L_l6ct.obs, x="PC1", hue="condition", kind="kde")

# 获取FacetGrid中的Axes对象，使用g.axes[0, 0]来访问第一个子图的Axes对象
axes = g.axes[0, 0]

# 设置偏移量初始值
offset = 0  # 偏移量（百分比）

# 计算每个condition的FWHM并添加到图中
for condition in condition_list:
    condition_data = V2L_l6ct.obs[V2L_l6ct.obs['condition'] == condition]['PC1']
    
    # 使用scipy的gaussian_kde计算KDE
    kde = gaussian_kde(condition_data, bw_method='silverman')  # 可以选择合适的带宽方法
    x = np.linspace(min(condition_data), max(condition_data), 1000)
    y = kde(x)
    
    # 找到最大值和半最大值
    max_value = np.max(y)
    half_max = max_value / 2
    
    # 找到半最大值的左右位置（即FWHM的边界）
    fwhm_indices = np.where(y >= half_max)[0]
    fwhm_left = x[fwhm_indices[0]]
    fwhm_right = x[fwhm_indices[-1]]
    
    # 计算FWHM
    fwhm = fwhm_right - fwhm_left
    
    # 将计算出的FWHM存入字典
    fwhm_dict[condition] = fwhm
    
    # 在图中添加FWHM标注
    axes.axvline(fwhm_left, color='gray', linestyle='--', linewidth=1)
    axes.axvline(fwhm_right, color='gray', linestyle='--', linewidth=1)
    
    # 获取y轴的范围
    ymin, ymax = axes.get_ylim()
    
    # 动态调整文本的纵坐标
    text_y_position = ymax * 0.9 - offset
    offset += 0.05  # 每次增加偏移量，避免重叠
    
    # 将文本的纵坐标设置为y轴的适当位置
    axes.text((fwhm_left + fwhm_right) / 2, text_y_position, f"FWHM: {fwhm:.2f}", 
              horizontalalignment='center', color='black')

# 将计算出的FWHM值赋值到V2L_l6ct.obs['FWHM']中
V2L_l6ct.obs['FWHM'] = V2L_l6ct.obs['condition'].map(fwhm_dict)

# 添加标题和标签
g.fig.suptitle('V2L L6CT', fontsize=16)
g.set_axis_labels('PC1', 'Density')
#plt.show()

# 显示保存了FWHM的DataFrame前几行（可选）
print(V2L_l6ct.obs[['condition', 'FWHM']].head())
output_path = "F:/spatial/V1_integration/NR_V/figures/V2L_l6ct_FWHM.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()

print(f"PDF saved as {output_path}")

      condition      FWHM
96904        NR  2.741406
96908        NR  2.741406
96938        NR  2.741406
96939        NR  2.741406
96944        NR  2.741406
PDF saved as F:/spatial/V1_integration/NR_V/figures/V2L_l6ct_FWHM.pdf
